In [114]:
from pathlib import Path
import pickle
import numpy as np
import pretty_midi

In [115]:
artists_songs_paths = {
    'Bach': ['data/orchestra/bach'],
    'Mozart': ['data/orchestra/mozart'],
    'Schumann x Chopin': ['data/orchestra/schumann', 'data/orchestra/chopin'],
    'Debussy x Albeniz': ['data/orchestra/debussy', 'data/orchestra/albeniz'],
    'All': ['data/orchestra']
}

artists_notes_paths = {
    'Bach': 'data/notes_files/bach_notes',
    'Mozart': 'data/notes_files/mozart_notes',
    'Schumann x Chopin': 'data/notes_files/schumann_chopin_notes',
    'Debussy x Albeniz': 'data/notes_files/debussy_albeniz_notes',
    'All': 'data/notes_files/all_notes'
}

In [116]:
artist_notes = {}

In [117]:
for artist, paths in artists_songs_paths.items():
    
    training_notes = []
    
    # if artists_notes_paths[artist] file already exists, skip this artist
    if Path(artists_notes_paths[artist]).exists():
        with open(artists_notes_paths[artist], 'rb') as filepath:
            training_notes = pickle.load(filepath)
        
        artist_notes[artist] = training_notes
        
        continue

    for path in paths:
        for count, midi_file in enumerate(Path(path).rglob('*.mid')):
                print(f"Parsing {midi_file}")
                notes = []
                durations = []

                try:
                    #parsing MIDI files one by one
                    midi_data = pretty_midi.PrettyMIDI(str(midi_file))
                except:
                    print("Error parsing file: {}".format(midi_file))
                    continue

                #depending on the element found in the instrument. Like we would have 'rest' for drums,
                #chords for guitar
                instrument = midi_data.instruments[0] if len(midi_data.instruments) > 0 else None
                
                if instrument is None:
                    continue
                
                sorted_notes = sorted(instrument.notes, key=lambda x: x.start)
                
                chord = [instrument.notes[0]]
                chord_end = instrument.notes[0].end
                first = True
                for note in sorted_notes:
                    if first:
                        first = False
                        continue
                    
                    if note.start > chord_end:
                        notes.append(chord)
                        chord = []
                        chord_end = note.end
                        chord.append(note)
                    else:
                        chord.append(note)
                        chord_end = note.end if note.end > chord_end else chord_end
                
                #notes and duration hold seuence for one music piece. 
                training_notes.append(notes)
    
    artist_notes[artist] = training_notes

    with open(artists_notes_paths[artist], 'wb') as filepath:
        pickle.dump(training_notes, filepath)


Parsing data\orchestra\bach\bach_846.mid
Parsing data\orchestra\bach\bach_847.mid
Parsing data\orchestra\bach\bach_850.mid
Parsing data\orchestra\mozart\mz_311_1.mid
Parsing data\orchestra\mozart\mz_311_2.mid
Parsing data\orchestra\mozart\mz_311_3.mid
Parsing data\orchestra\mozart\mz_330_1.mid
Parsing data\orchestra\mozart\mz_330_2.mid
Parsing data\orchestra\mozart\mz_330_3.mid
Parsing data\orchestra\mozart\mz_331_1.mid
Parsing data\orchestra\mozart\mz_331_2.mid
Parsing data\orchestra\mozart\mz_331_3.mid
Parsing data\orchestra\mozart\mz_332_1.mid
Parsing data\orchestra\mozart\mz_332_2.mid
Parsing data\orchestra\mozart\mz_332_3.mid
Parsing data\orchestra\mozart\mz_333_1.mid
Parsing data\orchestra\mozart\mz_333_2.mid
Parsing data\orchestra\mozart\mz_333_3.mid
Parsing data\orchestra\mozart\mz_545_1.mid
Parsing data\orchestra\mozart\mz_545_2.mid
Parsing data\orchestra\mozart\mz_545_3.mid
Parsing data\orchestra\mozart\mz_570_1.mid
Parsing data\orchestra\mozart\mz_570_2.mid
Parsing data\orch

In [118]:
class Notes:
    def __init__(self, notes):
        self.notes = notes
        notes = sorted(notes, key=lambda x: x.start)

        # first note is the one with the earliest start time
        self.first_note = notes[0]
        self.start = self.first_note.start

        notes = sorted(notes, key=lambda x: x.end)
        #last note is the one with the latest end time
        self.last_note = notes[-1]
        self.end = self.last_note.end
        self.duration = self.end - self.start
        self.parse_notes()
        
    def parse_notes(self):
        for note in self.notes:
            note.start -= self.start
            note.end -= self.start
            
    def __repr__(self) -> str:
        return f"Notes: len={len(self.notes)}, start={self.start}, end={self.end}, duration={self.duration}"
    
    def get_last_notes(self, n):
        if len(self.notes) < n:
            return self.notes
        return self.notes[-n:]

In [119]:
artist_notes = {key: [[Notes(notes) for notes in song] for song in training_notes] for key, training_notes in artist_notes.items()}

In [120]:
artist_notes = {key:[item for sublist in training_notes for item in sublist] for key, training_notes in artist_notes.items()}


In [121]:
def make_previous_tuple(notes):
    for i in range(len(notes)-1):
        yield ((notes[i], i), 
               (notes[i+1], i+1))
        
previous_artist_notes = {key:make_previous_tuple(training_notes) for key, training_notes in artist_notes.items()}

In [122]:
artist_note_dicts = {}

for artist, previous_notes in previous_artist_notes.items():
    
    note_dict = {}

    for (note_1, _), (note_2, idx) in previous_notes:
        key = (note_1.last_note.pitch)
        if key in note_dict.keys():
            note_dict[key].append((note_2, idx))
        else:
            note_dict[key] = [(note_2, idx)]
            
    artist_note_dicts[artist] = note_dict

In [123]:
artist_note_dicts['Bach']

{76: [(Notes: len=3, start=1.0111756895833335, end=1.6192839395833336, duration=0.6081082500000001,
   1),
  (Notes: len=3, start=2.0246894395833337, end=2.612527414583334, duration=0.5878379750000002,
   2),
  (Notes: len=3, start=2.6327751666666668, end=3.2327976895833337, duration=0.6000225229166669,
   3),
  (Notes: len=3, start=3.6382031895833338, end=4.211264789583333, duration=0.5730615999999991,
   4),
  (Notes: len=3, start=10.542302737499996, end=11.141121487499996, duration=0.5988187499999995,
   13),
  (Notes: len=3, start=11.535858487499997, end=12.10363028958333, duration=0.5677718020833336,
   14),
  (Notes: len=3, start=12.122903237499996, end=12.71505318958333, duration=0.592149952083334,
   15),
  (Notes: len=3, start=13.12045868958333, end=13.693520289583331, duration=0.5730616000000008,
   16)],
 77: [(Notes: len=3, start=4.230537737500001, end=4.822654129166667, duration=0.5921163916666661,
   5),
  (Notes: len=3, start=5.220005129166666, end=5.784298679166667, dur

In [124]:
chains = {key: [(artist_notes[key][0], 0)] for key in artist_notes.keys()}
n_words = 500

In [125]:
chains

{'Bach': [(Notes: len=3, start=0.4070946895833334, end=0.9912745791666667, duration=0.5841798895833333,
   0)],
 'Mozart': [(Notes: len=3, start=0.0, end=0.42304200000000003, duration=0.42304200000000003,
   0)],
 'Schumann x Chopin': [(Notes: len=160, start=0.75, end=40.8277393333333, duration=40.0777393333333,
   0)],
 'Debussy x Albeniz': [(Notes: len=95, start=0.12211000000000001, end=11.671070560416666, duration=11.548960560416667,
   0)],
 'All': [(Notes: len=18, start=0.321096, end=6.813055, duration=6.4919590000000005,
   0)]}

In [126]:
len(artist_notes['Bach'])

161

In [131]:
complete_chains = {}
for artist, chain in chains.items():
    complete_chain = chain.copy()
    for i in range(n_words):
        notes_1, idx = complete_chain[-1]
        prob = np.random.random()
        if prob < 0.5 or (notes_1.last_note.pitch) not in artist_note_dicts[artist].keys():
            if idx + 1 >= len(artist_notes[artist]):
                complete_chain.append((Notes([pretty_midi.Note(0, 0, 0, 1)]), -1))
                continue
            new_note = artist_notes[artist][idx + 1]
            complete_chain.append((new_note, idx + 1))
            continue
        
        selected = artist_note_dicts[artist][notes_1.last_note.pitch]
        idx = np.random.choice(len(selected))
        complete_chain.append(selected[idx])
    
    complete_chains[artist] = complete_chain


In [142]:
complete_chains['All']

[(Notes: len=18, start=0.321096, end=6.813055, duration=6.4919590000000005, 0),
 (Notes: len=23, start=7.110205666666666, end=12.834832333333331, duration=5.724626666666665,
  1),
 (Notes: len=42, start=13.149153916666664, end=26.24811591666667, duration=13.098962000000006,
  2),
 (Notes: len=3, start=26.464332250000002, end=26.770845583333337, duration=0.30651333333333497,
  3),
 (Notes: len=3, start=26.941058250000005, end=27.24125108333334, duration=0.3001928333333339,
  4),
 (Notes: len=3, start=68.02060387291667, end=68.11361562291667, duration=0.09301175000000228,
  1950),
 (Notes: len=2, start=630.9633251270816, end=631.450593710415, duration=0.48726858333338896,
  55158),
 (Notes: len=3, start=53.645407750000004, end=54.631600875000004, duration=0.9861931249999998,
  40285),
 (Notes: len=4, start=47.036556802083304, end=47.214746302083306, duration=0.178189500000002,
  14608),
 (Notes: len=2, start=47.274142802083304, end=47.408143052083304, duration=0.1340002499999997,
  14609

In [132]:
for artist, chain in complete_chains.items():
    offset = 0
    # Create a PrettyMIDI object
    chord = pretty_midi.PrettyMIDI()
    program = pretty_midi.instrument_name_to_program('Orchestral Harp')
    instrument = pretty_midi.Instrument(program=program)

    for notes_, _ in chain:
        for new_note in notes_.notes:
            note = pretty_midi.Note(
                velocity=new_note.velocity, pitch=new_note.pitch, start=offset+new_note.start, end=offset+new_note.end)
            instrument.notes.append(note)

        offset += notes_.duration + 0.1

    chord.instruments.append(instrument)
    # Write out the MIDI data
    chord.write(f"test_{artist}_harp.mid")